# Robot dance: a city-wise automatic control of Covid-19 mitigation levels

This is an automatic control system that considers working mobility between different cities and a control framework that suggests the level of protective measures, in order to avoid the collapse of the health system. 
The model is the dollowing: let a graph with $K$ cities

$$
\hat{S}_i + \hat{E}_i + \hat{I}_i + \hat{R}_i = N_i, i=1,\dots,K.
$$

Let $S_{ij}$ be the number of susceptible individuals in the population that leave city $i$ to work at city $j$.

The effective population in city $i$, **during the day** is 

$$
N_i^{\mathrm{eff}} = \hat{S}_{ii} + \hat{E}_{ii} + \hat{I}_{ii} + \hat{R}_{ii} + U_i^{in},
$$

such that $U_i^{in}$ is the number of individuals that travels from The other cities. 
Therefore, 

$$
U_i^{in} = \sum_{j=1}^n \hat{S}_{ji} + \hat{E}_{ji} + \hat{I}_{ji} + \hat{R}_{ji}.
$$

It is assumed that individuals of city $i$ that work at $j$ can only be infected there, during the day.
Thus

$$
\frac{d}{dt}\hat{S}_{ii} = -\frac{r_i}{N_i^{\mathrm{eff}}}\frac{1}{T_{\mathrm{inf}}}\hat{S}_{ii}\hat{I}_{ii} - \kappa\frac{r_i}{N_i^{\mathrm{eff}}}\frac{1}{T_{\mathrm{inf}}}\sum_{j=1}^K \hat{S}_{ii}\hat{I}_{ji},
$$
where $\kappa$ is the rate of infected individuals. 
Considering $\kappa = 0$,

$$
\frac{d}{dt}\hat{S}_{ij} = -\frac{r_j}{N_j^{\mathrm{eff}}}\frac{1}{T_{\mathrm{inf}}}\hat{S}_{ij}\hat{I}_{jj} 
$$

It is considered the normalized version
$$
X_i = \frac{\hat{X}_i}{N_i}, X \in \{S, E, I,  R\}
$$
and
$$
\dot{S}_i = -\frac{1}{T_{\mathrm{inf}}} \sum_{j=1}^n \frac{r_j}{P_j^{\mathrm{eff}}} S_{ij} I_{jj},
$$
where
$$
P_j^{\mathrm{eff}} = \sum_{i=1}^K \frac{N_i}{N_j}(S_{ij} + E_{ij} + I_{ij} + R_{ij})
$$
The fraction $S_{ij}$ can be estimated through $S_{ij} = S_i p_{ij}$, where $p_{ij}$ is the daily accumulated percentage of inhabitants of $i$ that travels to $j$.

During the night, the works go home and the model reduces to the standard SEIR.

The equations are, therefore, 

$$
\begin{split}
\frac{dS_i}{dt} &= -\alpha(t)\left(\frac{S_i}{T_{\mathrm{inf}}}\sum_{j=1}^K \frac{r_j(t)}{P_j^{\mathrm{eff}}} p_{ij} I_{jj}\right) - (1-\alpha(t))\left(\frac{r_i(t)}{T_{\mathrm{inf}}}S_i I_i\right) \\
\frac{dE_i}{dt} &= \alpha(t)\left(\frac{S_i}{T_{\mathrm{inf}}}\sum_{j=1}^K \frac{r_j(t)}{P_j^{\mathrm{eff}}} p_{ij} I_{jj}\right) + (1-\alpha(t))\left(\frac{r_i(t)}{T_{\mathrm{inf}}}S_i I_i\right) - \frac{1}{T_{\mathrm{inc}}}E_i \\
\frac{dI_i}{dt} &= \frac{1}{T_{\mathrm{inc}}}E_i - \frac{1}{T_{\mathrm{inf}}}I_i \\
\frac{dR_i}{dt} &= \frac{1}{T_{\mathrm{inf}}}I_i,
\end{split}
$$

where $T_{\mathrm{inc}}$ is the incubation period and $T_{\mathrm{inf}}$ is the recovery time.
We can say that $r_i(t) = \beta_i(t)/\gamma = \beta_i(t) T_{\mathrm{inf}}$.


The variable $r_j(t)$ is the control variable that means the reproductive number at time $t$ at city $j$.
They minimize a function changing this function. 
Moreover, the variable $\alpha(t) = 1$ during the working hours and zero otherwise.

In [1]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import sympy as sp
from scipy.sparse.linalg import eigs

Model code

In [ ]:
def model_ode(t, y, beta, gamma, eta, A, u, N_cities): 
    s_dot, x_dot, r_dot = np.zeros(N_cities), np.zeros(N_cities), np.zeros(N_cities)
    s, x, r = y[:N_cities], y[N_cities:2*N_cities], y[2*N_cities:]
    for i in range(N_cities):
        s_dot[i] = -beta[i] * s[i] * x[i] - eta[i] * s[i] + (A[i,:]*eta*s).sum() - u(t)[i] * s[i]
        x_dot[i] = beta[i] * s[i] * x[i] - eta[i] * x[i] + (A[i,:]*eta*x).sum() - gamma * x[i]
        r_dot[i] = gamma * x[i] + u(t)[i] * s[i] - eta[i] * r[i] + (A[i,:]*eta*r).sum()
    return np.hstack([s_dot, x_dot, r_dot])